In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sae_lens.saes.sae import SAE
from transformer_lens import HookedTransformer
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from sae_lens import SAE, ActivationsStore

In [ ]:
torch.set_grad_enabled(False)
model = HookedTransformer.from_pretrained("gemma-2b-it")
sae, cfg, sparsity  = SAE.from_pretrained(
  "gemma-2b-it-res-jb",
  "blocks.12.hook_resid_post"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")   

In [ ]:
tokenizer.encode("Bruh I'm finna fart")

In [ ]:
my_strings = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is rapidly advancing.",
    "Learning about SAEs is fascinating and complex.",
    "Hello world from a simple string!",
] * 5000

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return {"text": self.texts[idx]}

# Create DataLoader
dataset = TextDataset(my_strings)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
activation_store = ActivationsStore.from_sae(
    model=model,
    sae=sae,
    # tokenizer=tokenizer,  # Explicit tokenizer
    dataset=dataloader,
    streaming=False,
    train_batch_size_tokens=16,
    n_batches_in_buffer=1,
    device='cpu'
)

In [ ]:

# Process activations
reconstructions = []
for batch in activation_store:
    # SAE returns: (reconstruction, feature_acts, loss, ...)
    with torch.no_grad():
        output = sae(batch)
    reconstructions.append(output[0].cpu())  # Store on CPU

# Combine results (optional)
all_recons = torch.cat(reconstructions, dim=0)
print("Reconstructions shape:", all_recons.shape)

---


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformer_lens import HookedTransformer
from sae_lens import ActivationsStore, SAE
from transformers import AutoTokenizer

torch.set_grad_enabled(False)

In [2]:

torch.set_grad_enabled(False)
model = HookedTransformer.from_pretrained("gemma-2b-it")
sae, cfg, sparsity  = SAE.from_pretrained(
  "gemma-2b-it-res-jb",
  "blocks.12.hook_resid_post"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer


/var/folders/td/xbjvmnz50_z9p_v_y9frpj3c0000gn/T/ipykernel_783/2709055621.py:3: DeprecationWarning: Unpacking SAE objects is deprecated. SAE.from_pretrained() now returns only the SAE object. Use SAE.from_pretrained_with_cfg_and_sparsity() to get the config dict and sparsity as well.
  sae, cfg, sparsity  = SAE.from_pretrained(


In [14]:
# Dataset with repeated strings
my_strings = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is rapidly advancing.",
    "Learning about SAEs is fascinating and complex.",
    "Hello world from a simple string!",
] * 5000  # 20,000 examples

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return {"text": self.texts[idx]}

# Create DataLoader
dataset = TextDataset(my_strings)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [15]:
# Initialize ActivationsStore
activation_store = ActivationsStore.from_sae(
    model=model,
    sae=sae,
    # tokenizer=tokenizer,  # Explicit tokenizer
    dataset=dataloader,
    streaming=True,
    train_batch_size_tokens=16,
    n_batches_in_buffer=1,
    device='cpu'
)

/Users/pardan/Code/SAE/sae-env/lib/python3.11/site-packages/sae_lens/training/activations_store.py:317: UserWarning: Dataset is not tokenized. Pre-tokenizing will improve performance and allows for more control over special tokens. See https://jbloomaus.github.io/SAELens/training_saes/#pretokenizing-datasets for more info.
  warnings.warn(


In [16]:


max_batches = 2  # Process only 100 batches
reconstructions = []

for i, batch in enumerate(activation_store):
    print(f"Batch no: {batch}")
    if i >= max_batches:
        break
    with torch.no_grad():
        output = sae(batch)
    reconstructions.append(output[0].cpu())

# Combine results (optional)
all_recons = torch.cat(reconstructions, dim=0)
print("Reconstructions shape:", all_recons.shape)